In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: auto; }</style>"))
# TODO need to ctrl-shift-I after that?? wtf?

In [ ]:
import os
from pathlib import Path
pp = str(Path(os.getcwd()).absolute().parent / 'src')
import sys
sys.path.insert(0, pp)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
from bokeh.models import LinearAxis, Range1d
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource as CDS
import pandas as pd
from importlib import reload
from dashboard.core.bokeh import rolling
import matplotlib.pyplot as plt

In [ ]:
%autoreload 2
# TODO hmm, maybe this 2 is useful to reload submodules? check later
# yeah, without it seems to throw some crappy errors?

In [ ]:
from dashboard.sleep import _plot_all_sleep as P
from dashboard.data import sleep_dataframe as DF

In [ ]:
odf = DF()

In [ ]:
from dashboard.core.pandas import lag_df, unlocalize
df = odf[odf['error'].isna()]
df = df.set_index('date').sort_index()
# df = df.iloc[441:]

df.index = pd.to_datetime(df.index)
# print(df)
df = df.resample('D').mean()
# print(df)

col = 'avg_hr'
# col = 'hrv_morning'
# col = 'hrv_evening'
# col = 'avg_temp'
hr = df[col].dropna()


# plt.figure(figsize=(30,10))
# autocorrelation_plot(hr)
# plt.show()

res = pd.DataFrame()
for fr in range(750):
    # TODO ugh. not sure what's the terminology should be? ahead or behind??
    sh = hr.shift(-fr, freq='D')
    sh.name = sh.name + f'+{fr}'
    xxx = pd.DataFrame(index=hr.index)
    xxx[hr.name] = hr
    xxx = xxx.merge(sh, how='outer', left_index=True, right_index=True)
    # todo timedelta?
    color = 'red' if len(xxx.dropna()) < 100 else 'blue'
    res.loc[fr, hr.name] = xxx.corr().iloc[0, 1]
    res.loc[fr, 'color'] = color
# print(res)
# print(hr.autocorr(1)) # TODO what units is lag?
# res.plot()
# TODO wow. crazy seasonality! (weeks??)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
ss = res[col].resample('D').mean().interpolate('linear')
dec = seasonal_decompose(x=ss)# , period=365)
size = (30, 5)
# TODO wonder how it calculates seasonal? also it's not catching yearly trends?
# ddd = dec.seasonal + dec.trend + dec.resid
dec.seasonal.plot(figsize=size, color='red')
dec.trend   .plot(figsize=size, color='blue')
dec.resid   .plot(figsize=size, color='green')

In [ ]:
zzz = list(res[col])


from dashboard.core.analysis import fourier, periods
from dashboard.core.bokeh import date_figure

# print(fourier(zzz))


zzz = list(hr) # TODO ok, here def need to resample??
print(fourier(zzz))
print(periods(zzz, n=5))
# TODO function to residualize?
# TODO hmmm. not sure if should process the autocorrelations or the original signal???
# fewfwf

res = df
res = res.dropna()
res['color'] = 'blue'

# TODO need some sort of quickplot..
p = date_figure(df=res)
p.line   (source=CDS(res), x='date', y=col)
p.scatter(source=CDS(res), x='date', y=col, color='color', size=2)
p.title.text = f'Autocorrelaction of {col}'
p.yaxis.axis_label = col
p.xaxis.axis_label = 'days'
show(p)

# TODO how to residualize??

# TODO https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

# TODO ugh. lag plot code seems to use .values (hence, indexes, not dates?)
# pd.plotting.lag_plot(hr, lag=1)
# from pandas.plotting import autocorrelation_plot
# TODO same with autocorr plot.. need to fill gaps with nans?

# TODO show how many points are participating in the lag?

# I guess I need my own function + copy stuff from 
# ~/.local/lib/python3.8/site-packages/pandas/plotting/_matplotlib/misc.py

# todo ok, this is interesting, but not sure. combine multiple?
# TODO fourier breakdown?
# TODO exclude jawbone data?
# TODO wow. nightly temp. correlation plot looks pretty stupid

# TODO https://en.wikipedia.org/wiki/Periodogram
# https://en.wikipedia.org/wiki/Seasonal_subseries_plot

In [ ]:
# todo onclick example https://stackoverflow.com/questions/53825387/python-bokeh-update-scatter-plot-colors-on-callback
# TODO determine delta hr and plot 'dependence' on workdays?
# TODO or plot 'derivative'?
# todo disable individual points by default?

In [ ]:
# TODO that probably doesn't mean a thing...
# TODO hmm pandas also has nice stuff
# core.window.ExponentialMovingWindow.cov
#    Exponential weighted sample covariance.
#core.window.Expanding.cov
#    Expanding sample covariance.
#core.window.Rolling.cov
#    Rolling sample covariance.


# rs = rolling(x='dt', y='sleepy', df=s, avgs=[])
# rc = rolling(x='dt', y='kcal'  , df=c, avgs=[], color='green', context=rs)

# todo hmm. maybe display a bar band after delta? not sure..
# show(rc.figure)l

# for each moment, find stuff within the delta??
# TODO eh, this doesn't make much sense.. just plot the correlation directly?

In [ ]:
import my.hypothesis as H
hls = H.highlights()

In [ ]:
from dashboard.data import cross_trainer_dataframe as DF
df = DF()

In [ ]:
from dashboard.cardio import *
show(plot_cross_trainer(df))

In [ ]:
df = pd.DataFrame(h._asdict() for h in hls)
# print(df)


df['count'] = 1 # meh

df['date'] = df['created'].dt.date

ddf = df.groupby(['date'], as_index=False).sum()


p = date_figure()
add_daysoff(p, dates=ddf['date'])
p.scatter(x='created', y='count'  , source=CDS(data=df ), color='blue', legend_label='Hypothesis')
p.vbar   (x='date'   , top='count', source=CDS(data=ddf))


p.legend.click_policy = 'hide'

show(p)

# TODO might be nice to have a slider through a week at a time or something?
# TODO maybe could display other sources on the same plots?

In [ ]:
# TODO hmm, maybe update regression lines dynamically? Wonder if there is as callback in bokeh
# https://stackoverflow.com/q/53287715/706389 here some sample code

In [ ]:
# regression tap? some callbacky thing
# http://holoviews.org/reference/streams/bokeh/regression_tap.html

# TODO dynamic regression? https://gist.github.com/darothen/b1538c6571b67ed610669314551d3356

xxx = emfit_df[['avg_hr', 'coverage']].dropna()

# normal.opts(size=10) * hv.Slope.from_scatter(normal)
sc = hv.Scatter((xxx['avg_hr'], xxx['coverage']))
# TODO shit. need to filter nans??
hv.Slope.from_scatter(sc).opts(color='red') * sc.opts(size=10)

In [ ]:
from dashboard.sleep import plot_sleep_correlations
plot_sleep_correlations(emfit_df)

In [ ]:
# some examples of server with interactive stuff? 
# https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/main.py

# these for sleep intervals? https://docs.bokeh.org/en/latest/docs/gallery/stacked_area.html
# also I had some old jawbone plots I think...
# also existing emfit plots

# food plots (breakdown) https://docs.bokeh.org/en/latest/docs/gallery/bar_stacked.html

# TODO errors should be aggregated and displayed as a separate column? with ability to toggle


# TODO ok, plotly has some callbacks for lasso/rectangle selection https://plotly.com/python/v3/selection-events/
# with these, I could dynamically update the regression lines, I guess


# TODO some nice dynamic interactions https://docs.bokeh.org/en/0.10.0/docs/user_guide/interaction.html

In [ ]:
from dashboard.data import sleepiness_dataframe as SDF
sdf = SDF()

In [ ]:
from dashboard.sleepiness import plot_sleepiness_vs_sleep
show(plot_sleepiness_vs_sleep())

In [ ]:
from dashboard.data import locations_dataframe
locs = locations_dataframe()

In [ ]:
import pandas as pd
import numpy as np
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
from bokeh.io import output_notebook
output_notebook()
pd.set_option('display.max_columns', 100)
gv.extension('bokeh')

In [ ]:
from datetime import date
from_ = date(2019, 2, 1)
to    = date(2019, 4, 1)
df = locs[(from_ <= locs.index.date) & (locs.index.date <= to)] # meh, but not too bad

df = df[['lat', 'lon']]
df['date'] = df.index.date # meh
df[:5]
# todo wonder, what's its performance? how many points it can handle?
# ok, 2 month of data took about a minute to render

In [ ]:
# https://malouche.github.io/notebooks/geoviews2.html
# https://data-dive.com/interactive-maps-made-easy-geoviews
plot = (gf.ocean * gf.land * gf.coastline * gf.borders).options(
    'Feature', projection=crs.Mercator(), global_extent=True, frame_height=1000,frame_width=1500,
)
# Convert locations to points and overlay on previous map
points = gv.Points(df, kdims=['lon', 'lat'], vdims=['date']).groupby(['date'])
plot_points = plot * points.opts(alpha=0.3, color='red', size=4)
plot_points.opts(width=500, height=400)
# todo hmm, how to set initial zoom? ideally, auto adjust it interactively depending on the day
# todo how to add proper keybindings for move/zoom? with ctrl etc